<a href="https://colab.research.google.com/github/Djaxis/MY-Python-Evolution/blob/main/COLLECT_DATA_STATION_VELO_NANTES_TOUTES_LES_180_MINUTES_Brice_De_Campos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls /content/drive/MyDrive/BDD/

DataNantes2023.csv  UENG_metadata.csv  UENG_metadata.gsheet


In [ ]:
from google.colab import drive
import os.path
import pandas as pd
import csv
import json
import numpy as np
import plotly.express as px
import requests
import seaborn as sns
import time
import urllib.request
import matplotlib.pyplot as plt


premierTour = True #À la fin de chaque boucle, définissez premierTour sur False.
while True : 
    # monter votre Google Drive
    drive.mount('/content/drive')
    # vérifier si le dossier 'BDD' existe dans votre Google Drive
    bdd_path = '/content/drive/MyDrive/BDD'
    if not os.path.exists(bdd_path):
            os.mkdir(bdd_path)

    # vérifier si le fichier 'DataNantes2023.csv' existe dans le dossier 'BDD' de votre Google Drive
    csv_path = '/content/drive/MyDrive/BDD/DataNantes2023.csv'
    if not os.path.exists(csv_path):
            # si le fichier n'existe pas, créer un nouveau dataframe avec le nom "DataNantes2023"
            DataNantes2023 = pd.DataFrame()
            # enregistrer le dataframe dans le fichier CSV
            DataNantes2023.to_csv(csv_path, index=False)
            print(f'Le fichier {csv_path} a été créé.')
    else:
            print(f'Le fichier {csv_path} existe déjà.')



    link = 'https://data.nantesmetropole.fr/api/records/1.0/search/?dataset=244400404_stations-velos-libre-service-nantes-metropole-disponibilites&q=&facet=banking&facet=bonus&facet=status&facet=contract_name'
    r = requests.get(link)
    data = json.loads(r.text)

    df_new = pd.json_normalize(data['records']) # pour comprendre et prendre la partie de la data souhaitée. Puis on normalise pour avoir un tableau
    df_new = df_new .reset_index(drop=True)
    dfviz= df_new[['fields.status','fields.name','fields.address','fields.available_bikes','fields.available_bike_stands','fields.bike_stands','fields.last_update','fields.position']]
    # pour voir les colonnes qui me sont nécessaires et les placer dans un ordre précis
    dfviz = dfviz.reset_index(drop=True)
    #renommer les colonnes
    dfrenam = dfviz.rename(columns={'fields.status':'Status',
                                'fields.name':'Nom Station',
                                'fields.address':'Adresse De Station',
                                'fields.available_bikes':'Vélos Disponibles',
                                'fields.available_bike_stands':'Socles Disponbles',
                                'fields.bike_stands':'Nombre de Socles',
                                'fields.last_update':'Dernière Mise à Jour',
                                'fields.position':'Localisation',
                                })

    #création de la ligne taux de remplissage
    dfrenam['taux de remplissage'] = dfrenam['Vélos Disponibles']/dfrenam['Nombre de Socles']*100
    # Arrondir la colonne 'taux de remplissage' à deux décimales
    dfrenam['taux de remplissage'] = dfrenam['taux de remplissage'].round(2)
        
    # Convertir la colonne en type datetime
    dfrenam['Dernière Mise à Jour'] = pd.to_datetime(dfrenam['Dernière Mise à Jour'], format='%Y-%m-%dT%H:%M:%S+00:00')
    # Séparer la colonne en deux colonnes distinctes date et heure
    dfrenam['date'] = dfrenam['Dernière Mise à Jour'].dt.date
    dfrenam['heure'] = dfrenam['Dernière Mise à Jour'].dt.time
    dfrenam.sort_values(by=['heure'], ascending=True)


    #Freqence: compte le nbre de fois qu'apparait une station dans la colonne.
    dfrenam['Frequence']= (len(dfrenam)) #dfrenam['Nom Station'].(len) rows = len(df.axes[0])      
    
    # Diviser la colonne "Nom Station" en deux parties en utilisant le séparateur "-"
    dfrenam[['NumStation','Nom Station']] = dfrenam['Nom Station'].str.split('-', expand=True)


    #réorganiser l'ordre des colonnes
    dfrenam = dfrenam.reindex(columns=['status','NumStation','Nom Station','heure', 'Adresse De Station', 'Vélos Disponibles',
       'Socles Disponbles', 'Nombre de Socles', 'Dernière Mise à Jour',
       'Localisation', 'taux de remplissage', 'date', 'Frequence'])
    

    #Dans l'ordre croissant 
    dfrenam = dfrenam.sort_values(by=['Nom Station', 'heure'])

    
    lire = pd.read_csv(r"/content/drive/MyDrive/BDD/DataNantes2023.csv",header =None ,sep = ",")
    lire = dfrenam.reset_index(drop=True)
    lire = pd.DataFrame(lire) #convertir le sheet en data 
    print(lire.columns)
    print("NOMBRE DE LIGNES PRESENTES:",len(lire))

    #la première boucle écrira les noms de colonnes, et les boucles suivantes ajouteront les nouvelles données au fichier existant sans écrire les noms de colonnes à nouveau.
    if premierTour:
        dfrenam.to_csv('/content/drive/MyDrive/BDD/DataNantes2023.csv', index=False, mode='w')
        dfrenam = dfrenam.reset_index(drop=True)
        premierTour = False

    
    else:
        dfrenam.to_csv('/content/drive/MyDrive/BDD/DataNantes2023.csv', index=False, mode='a', header=False)
        dfrenam = dfrenam.reset_index(drop=True)
        # vérification visuelle de la ré-écriture dans le fichier.
        lire = pd.read_csv(r"/content/drive/MyDrive/BDD/DataNantes2023.csv", header=0,sep = ",")
        lire = pd.DataFrame(lire) #convertir le sheet en data 

    print("NOMBRE DE LIGNES PRESENTES ACTUALISEES 180 SEC:",len(lire))
    print(lire)
    time.sleep(180)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Le fichier /content/drive/MyDrive/BDD/DataNantes2023.csv existe déjà.
Index(['status', 'NumStation', 'Nom Station', 'heure', 'Adresse De Station',
       'Vélos Disponibles', 'Socles Disponbles', 'Nombre de Socles',
       'Dernière Mise à Jour', 'Localisation', 'taux de remplissage', 'date',
       'Frequence'],
      dtype='object')
NOMBRE DE LIGNES PRESENTES: 10
NOMBRE DE LIGNES PRESENTES ACTUALISEES 180 SEC: 10
   status NumStation            Nom Station     heure  \
0     NaN     01011     BORNE TEST NANTES 1  14:41:50   
1     NaN     01012     BORNE TEST NANTES 2  11:08:09   
2     NaN        050                CHÂTEAU  15:07:49   
3     NaN        062  GARE DE NANTES NORD 2  15:06:27   
4     NaN        057         GAËTAN RONDEAU  15:07:23   
5     NaN        002         HÔTEL DE VILLE  15:07:13   
6     NaN        081                 MANGIN  15:09:11

In [ ]:
# Boucle pour exécuter le code toutes les 5 minutes pendant 24 heures
temps_total = 24 * 60 # 24 heures * 60 minutes
duree_attente = 5 # minutes
n = temps_total // duree_attente

for i in range(n):
  print(f'Exécution numéro {i+1}...')
  # Attendre la durée d'attente avant d'exécuter à nouveau le code
  time.sleep(duree_attente * 60)